# Part 1

Your raft makes it to the tropical island; it turns out that the small crab was an excellent navigator. You make your way to the resort.

As you enter the lobby, you discover a small problem: the floor is being renovated. You can't even reach the check-in desk until they've finished installing the new tile floor.

The tiles are all hexagonal; they need to be arranged in a hex grid with a very specific color pattern. Not in the mood to wait, you offer to help figure out the pattern.

The tiles are all white on one side and black on the other. They start with the white side facing up. The lobby is large enough to fit whatever pattern might need to appear there.

A member of the renovation crew gives you a list of the tiles that need to be flipped over (your puzzle input). Each line in the list identifies a single tile that needs to be flipped by giving a series of steps starting from a reference tile in the very center of the room. (Every line starts from the same reference tile.)

Because the tiles are hexagonal, every tile has six neighbors: east, southeast, southwest, west, northwest, and northeast. These directions are given in your list, respectively, as `e`, `se`, `sw`, `w`, `nw`, and `ne`. A tile is identified by a series of these directions with no delimiters; for example, esenee identifies the tile you land on if you start at the reference tile and then move one tile east, one tile southeast, one tile northeast, and one tile east.

Each time a tile is identified, it flips from white to black or from black to white. Tiles might be flipped more than once. For example, a line like esew flips a tile immediately adjacent to the reference tile, and a line like nwwswee flips the reference tile itself.

Here is a larger example:
```
sesenwnenenewseeswwswswwnenewsewsw
neeenesenwnwwswnenewnwwsewnenwseswesw
seswneswswsenwwnwse
nwnwneseeswswnenewneswwnewseswneseene
swweswneswnenwsewnwneneseenw
eesenwseswswnenwswnwnwsewwnwsene
sewnenenenesenwsewnenwwwse
wenwwweseeeweswwwnwwe
wsweesenenewnwwnwsenewsenwwsesesenwne
neeswseenwwswnwswswnw
nenwswwsewswnenenewsenwsenwnesesenew
enewnwewneswsewnwswenweswnenwsenwsw
sweneswneswneneenwnewenewwneswswnese
swwesenesewenwneswnwwneseswwne
enesenwswwswneneswsenwnewswseenwsese
wnwnesenesenenwwnenwsewesewsesesew
nenewswnwewswnenesenwnesewesw
eneswnwswnwsenenwnwnwwseeswneewsenese
neswnwewnwnwseenwseesewsenwsweewe
wseweeenwnesenwwwswnew
```

In the above example, 10 tiles are flipped once (to black), and 5 more are flipped twice (to black, then back to white). After all of these instructions have been followed, a total of 10 tiles are black.

Go through the renovation crew's list and determine which tiles they need to flip. After all of the instructions have been followed, how many tiles are left with the black side up?

In [1]:
from enum import Enum


def get_input():
    with open('inputs/24') as f:
        return f.read().splitlines()


# Translate the steps to unit vectors in the coordinate system where axis x corresponds
# to the E-W direction (growing in the direction of East) and axis y corresponds to the SW-NE
# direction (growing in the direction of NE).
STEPS = {
    'w': (-1, 0),
    'e': (1, 0),
    'sw': (0, -1),
    'ne': (0, 1),
    'se': (1, -1),
    'nw': (-1, 1)
}
    
def follow_steps(line):
    x = y = 0
    curr_step = ''
    for ch in line:
        curr_step += ch
        if curr_step in STEPS:
            s_x, s_y = STEPS[curr_step]
            x += s_x
            y += s_y
            curr_step = ''
    return (x, y)


def result1():
    inp = get_input()
    black_tiles = set()
    for steps in inp:
        tile = follow_steps(steps)
        if tile in black_tiles:
            black_tiles.remove(tile)
        else:
            black_tiles.add(tile)
    return len(black_tiles)


result1()

388

# Part 2

The tile floor in the lobby is meant to be a living art exhibit. Every day, the tiles are all flipped according to the following rules:

- Any black tile with zero or more than 2 black tiles immediately adjacent to it is flipped to white.
- Any white tile with exactly 2 black tiles immediately adjacent to it is flipped to black.

Here, tiles immediately adjacent means the six tiles directly touching the tile in question.

The rules are applied simultaneously to every tile; put another way, it is first determined which tiles need to be flipped, then they are all flipped at the same time.

In the above example, the number of black tiles that are facing up after the given number of days has passed is as follows:
```
Day 1: 15
Day 2: 12
Day 3: 25
Day 4: 14
Day 5: 23
Day 6: 28
Day 7: 41
Day 8: 37
Day 9: 49
Day 10: 37

Day 20: 132
Day 30: 259
Day 40: 406
Day 50: 566
Day 60: 788
Day 70: 1106
Day 80: 1373
Day 90: 1844
Day 100: 2208
```

After executing this process a total of 100 times, there would be 2208 black tiles facing up.

How many tiles will be black after 100 days?

In [2]:
from collections import defaultdict


def list_neighbors(tile):
    x, y = tile
    return [(x + step[0], y + step[1]) for step in STEPS.values()]


def get_initial_black_tiles():
    inp = get_input()
    black_tiles = set()
    for steps in inp:
        tile = follow_steps(steps)
        if tile in black_tiles:
            black_tiles.remove(tile)
        else:
            black_tiles.add(tile)
    return black_tiles
    

def count_black_neighs(black_tiles):
    num_black_neighs = defaultdict(int)
    for bt in black_tiles:
        for n in list_neighbors(bt):
            num_black_neighs[n] += 1
    return num_black_neighs


def flip_tiles(black_tiles):
    num_black_neighs = count_black_neighs(black_tiles)
    new_black_tiles = set()
    for bt in black_tiles:
        try:
            num_bl_neighs = num_black_neighs.pop(bt)
            if num_bl_neighs <= 2:
                new_black_tiles.add(bt)
        except KeyError:
            # The black tile has 0 black neighbors.
            pass
    for wt, num_bl_neighs in num_black_neighs.items():
        if num_bl_neighs == 2:
            new_black_tiles.add(wt)
    return new_black_tiles    


def result2():
    black_tiles = get_initial_black_tiles()
    for day in range(100):
        black_tiles = flip_tiles(black_tiles)
    return len(black_tiles)


result2()

4002